# Pack it

In [2]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
gribfile = "/Users/yylab/ikkifik/research/era5-reanalysis/_archives/ofunato_t2m_tp.grib"

In [ ]:
# xarray dataset open
t2m_dataset = xr.open_dataset(gribfile, engine="cfgrib")
tp_dataset = xr.open_dataset(gribfile, engine="cfgrib", backend_kwargs={'filter_by_keys': {'shortName': 'tp'}})

# xarray dataframe making
tp_df = tp_dataset.to_dataframe().reset_index()
t2m_df = t2m_dataset.to_dataframe().reset_index()
# convert temperature (t2m) kelvin to celcius
t2m_df['t2m'] = t2m_df['t2m'].apply(lambda c: c-273.15)
# join both separate dataframe into one dataframe
join = pd.merge(t2m_df, tp_df[['tp']], on=t2m_df.index, how='left')

# convert dataframe to geodataframe to obtain the geometry features
gdf = gpd.GeoDataFrame(
    join, geometry=gpd.points_from_xy(join.longitude, join.latitude), crs="EPSG:4326"
)

# Drop unused features, create validated datetime format
gdf = gdf.drop(columns=["number", "step", "time", "surface", "key_0", "latitude", "longitude"])
gdf['date'] = pd.to_datetime(gdf['valid_time']).dt.date
gdf['date'] = pd.to_datetime(gdf['date'], format='%Y-%m-%d')
gdf['time'] = pd.to_datetime(gdf['valid_time']).dt.time

skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/Users/yylab/miniconda3/envs/exp/lib/python3.12/site-packages/cfgrib/dataset.py", line 726, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/Users/yylab/miniconda3/envs/exp/lib/python3.12/site-packages/cfgrib/dataset.py", line 642, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1577872800, 1577876400, 1577880000, ..., 1763733600, 1763737200,
       1763740800], shape=(15064,))) new_value=Variable(dimensions=('time',), data=array([1577858400, 1577944800, 1578031200, ..., 1763532000, 1763618400,
       1763704800], shape=(2152,)))


,valid_time,t2m,tp,geometry,date,time
0,2020-01-01 10:00:00,-1.217017,1.349412e-04,POINT (141 39.505),2020-01-01,10:00:00
1,2020-01-01 10:00:00,-2.085181,1.009367e-04,POINT (141.2502 39.505),2020-01-01,10:00:00
2,2020-01-01 10:00:00,-2.485327,5.890802e-05,POINT (141.5004 39.505),2020-01-01,10:00:00
3,2020-01-01 10:00:00,-1.774146,2.278015e-05,POINT (141.7506 39.505),2020-01-01,10:00:00
4,2020-01-01 10:00:00,1.484521,4.749745e-06,POINT (142.0008 39.505),2020-01-01,10:00:00
5,2020-01-01 10:00:00,3.099634,9.536743e-07,POINT (142.251 39.505),2020-01-01,10:00:00
6,2020-01-01 10:00:00,-0.311987,8.388981e-05,POINT (141 39.255),2020-01-01,10:00:00
7,2020-01-01 10:00:00,-0.691870,5.519763e-05,POINT (141.2502 39.255),2020-01-01,10:00:00
8,2020-01-01 10:00:00,-0.754614,3.195927e-05,POINT (141.5004 39.255),2020-01-01,10:00:00
9,2020-01-01 10:00:00,0.060205,7.905066e-06,POINT (141.7506 39.255),2020-01-01,10:00:00


In [15]:
gdf.head(3)

,valid_time,t2m,tp,geometry,date,time
0,2020-01-01 10:00:00,-1.217017,0.000135,POINT (141 39.505),2020-01-01,10:00:00
1,2020-01-01 10:00:00,-2.085181,0.000101,POINT (141.2502 39.505),2020-01-01,10:00:00
2,2020-01-01 10:00:00,-2.485327,0.000059,POINT (141.5004 39.505),2020-01-01,10:00:00


In [13]:
# Do some grouping/aggregation based on the date
grouped_date = gdf.groupby("date", group_keys=True)[["tp", "t2m", "time", "geometry"]].apply(lambda x: x)
grouped_date

tp       t2m      time                 geometry
date                                                                        
2020-01-01 0       1.349412e-04 -1.217017  10:00:00       POINT (141 39.505)
           1       1.009367e-04 -2.085181  10:00:00  POINT (141.2502 39.505)
           2       5.890802e-05 -2.485327  10:00:00  POINT (141.5004 39.505)
           3       2.278015e-05 -1.774146  10:00:00  POINT (141.7506 39.505)
           4       4.749745e-06  1.484521  10:00:00  POINT (142.0008 39.505)
...                         ...       ...       ...                      ...
2025-11-21 361531  4.780633e-05  2.990137  16:00:00  POINT (141.2502 38.754)
           361532  3.175033e-05  4.247949  16:00:00  POINT (141.5004 38.754)
           361533  1.937119e-05  6.835107  16:00:00  POINT (141.7506 38.754)
           361534  2.968736e-06  8.128564  16:00:00  POINT (142.0008 38.754)
           361535  9.533539e-07  8.191797  16:00:00   POINT (142.251 38.754)

[361536 rows x 4 columns]

In [17]:
gdf_agg = gdf.groupby(by=[ "date", "geometry"]).agg(
    t2m_mean=('t2m', 'mean'), 
    tp_mean=('tp', 'mean')
)
gdf_agg

t2m_mean       tp_mean
date       geometry                                       
2020-01-01 POINT (141 38.754)       1.433758  2.507386e-05
           POINT (141.2502 38.754)  2.083939  1.129428e-05
           POINT (141.5004 38.754)  2.504646  6.188239e-06
           POINT (141.5004 39.005)  1.006808  1.376096e-05
           POINT (141.2502 39.005)  0.962863  3.360824e-05
...                                      ...           ...
2025-11-21 POINT (142.0008 39.005)  7.793604  4.599914e-06
           POINT (141.7506 39.005)  5.519678  2.558022e-05
           POINT (141.7506 38.754)  7.479464  1.423511e-05
           POINT (142.0008 38.754)  8.724756  1.552626e-06
           POINT (142.251 38.754)   8.864927  4.746868e-07

[51648 rows x 2 columns]

In [18]:
gdf_agg = gdf_agg.reset_index()
gdf_agg["t2m"] = gdf_agg[["t2m_mean"]]
gdf_agg["tp"] = gdf_agg[["tp_mean"]]

gdf_agg = gdf_agg.drop(columns=["t2m_mean", "tp_mean"])

In [20]:
gdf = gpd.GeoDataFrame(gdf_agg[['date', 't2m', 'tp']], geometry=gdf.geometry)
# gdf = gdf.to_crs({'init': 'epsg:4326'})
gdf = gdf.set_crs('epsg:4326')

In [21]:
gdf.head()

,date,t2m,tp,geometry
0,2020-01-01,1.433758,0.000025,POINT (141 39.505)
1,2020-01-01,2.083939,0.000011,POINT (141.2502 39.505)
2,2020-01-01,2.504646,0.000006,POINT (141.5004 39.505)
3,2020-01-01,1.006808,0.000014,POINT (141.7506 39.505)
4,2020-01-01,0.962863,0.000034,POINT (142.0008 39.505)


In [22]:
import numpy as np
np.unique(gdf[["date"]])

array(['2020-01-01T00:00:00.000000000', '2020-01-02T00:00:00.000000000',
       '2020-01-03T00:00:00.000000000', ...,
       '2025-11-19T00:00:00.000000000', '2025-11-20T00:00:00.000000000',
       '2025-11-21T00:00:00.000000000'],
      shape=(2152,), dtype='datetime64[ns]')

In [ ]:
import os
from datetime import datetime

# Turn into raster data
gdf["x"] = gdf["geometry"].x
gdf["y"] = gdf["geometry"].y

t2m = (gdf.set_index(["y", "x"]).t2m.to_xarray()).rio.set_crs(4326)
tp = (gdf.set_index(["y", "x"]).tp.to_xarray()).rio.set_crs(4326)

dir_path = os.path.join("temp_results", "raster")
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

t2m_path = os.path.join(dir_path, f"t2m_{datetime.strftime(gdf["date_min"][0], '%Y%m%d')}.TIF")
t2m.rio.to_raster(t2m_path)

tp_path = os.path.join(dir_path, f"tp_{datetime.strftime(gdf["date_min"][0], '%Y%m%d')}.TIF")
tp.rio.to_raster(tp_path)